# Overview

This notebook extracts unstructured data from Microsoft Outlook emails regarding Travel Waivers
that United Airlines sends out related to events/weather/etc. that may impact travel. 

The email data is cleaned and manipulated into a DataFrame that includes the Travel Waiver's
date sent, event name, severity level, cities impacted, and dates affected. 

The code can be broken down into the following 3 sections:

    1. Scrape Outlook to collect all Travel Waiver emails
    2. Create functions used to parse the email body text 
    3. Iterate over the Dictionary to create the cleaned dataset

Over time, the collection of this data may yield some interesting insights into events that impact travel.
Feel free to ask questions or leave suggestions/critique, I love to learn other's approaches to problems!

### The image below is an example of the email being scraped and my mailbox had over a year's worth of Travel Waiver emails 

![TravelWaiverImage](https://raw.githubusercontent.com/eli64s/Python-Email-Scraper/master/travel_waiver.PNG)

# Outlook Setup

In [1]:
import pandas as pd
import win32com.client
import re 

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6) # '6' refers to the index of a folder - in this case the Inbox Folder
messages = inbox.Items              # Get all items in the Inbox folder

### Retrieve Mailbox Messages

#### Iterate over the messages in the mailbox, storing the extracted data in a Dictionary 

In [2]:
travel_waivers = {} # Dictionary to store email data - send date, title, body text
                    # Iterate over messages, inserting email data belonging to SenderName - 'Travel Waivers'
                    # Try/Except statements are used as some emails in the mailbox have an unknown SenderName
key_val = 0
for message in messages:

    try:
        if message.SenderName == 'Travel Waivers':
            
            # Condition to check if a 'Travel Waivers' email is a reply and not an actual waiver
            if 'From:' in message.Body:
                pass
            
            else:
                # Dictionary to store the scraped data
                travel_waivers[key_val] = {
                    'Issue_Date': message.SentOn.strftime('%Y-%m-%d'),
                    'Event': None,
                    'Indicator': None,
                    'email_title': message.Subject,
                    'email_body': message.Body,
                    'City': None,
                    'Start_Date': None,
                    'End_Date': None,
                    'All_Dates': None
                }
                key_val += 1
            
    # An Exception occurs when the SenderName is unknown
    except Exception as error_message:
        print(error_message)

<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName
<unknown>.SenderName


# Create Functions to Clean & Extract Data from Email Body String

#### Function: Returns event name and severity indicator from the email title

In [3]:
def clean_title(text):
    '''
    Function that takes the email title and extracts the 
    Travel Waiver event title and severity indicator 
    '''
    
    # The event titles are uniform in each email and come after a closing parenthesis 
    title = text.split(')')[1]    
    title = title.split('-')[0]
    
    # All severity indicators start with '(S' - example) 'SEV1' 
    ind = text.split()
    ind = [i for i in ind if i.startswith('(S')][0]
    
    # Try/Except to remove the parentheses around indicator - example (SEV1)
    try:
        ind = ind.replace('(', '')
        ind = ind.replace(')', '')
    except:
        pass
    
    return title, ind
    

####  Function: Returns a substring between two words using regular expressions

In [4]:
def string_search(text, start_word, end_word):
    '''
    This function returns a substring between two words 
    using regular expressions
    '''
    
    # 'c' and 'd' are passed as placeholders to the function when called 
    # The first condition extracts the text related to the Travel Waiver's impacted cities 
    if start_word == 'c':
        start_word = re.findall(r"City:|Cities:", text)[0]                                 # Search text for City:
        end_word = re.findall(r"Travel date:|Travel date\(s\):|Travel dates:", text)[0]    # Search for 'Travel Dates:'

    # This condition extracts the text related to the Travel Waiver's date(s) 
    if end_word == 'd':
        start_word = re.findall(r"Travel date:|Travel date\(s\):|Travel dates:", text)[0]
        end_word = re.findall(', \d{4}', text)[0]             # Regex to find the ', YYYY'
        end_word = text.split(end_word, 1)[1]                 # Gets entire string after ', YYYY'

    # Use starting and ending word to extract substring relating to the data we want  
    substring_data = text[text.find(start_word) + len(start_word): text.rfind(end_word)]
   
    return substring_data

#### Function: Returns a list of the city codes in the given Travel Waiver 

In [5]:
def clean_cities(text):
    '''
    This function takes the substring of city codes
    that was created from the string_search() function  
    An example of a substring:
    '*    Cities: CHS / FAY / HHH / ILM / MYR / SAV    *'
    Function returns of a list of each 3 character city code
    '''
    
    text.strip('/')                         # Further clean city text - strip '/' that separates each city
    city_text = ''.join(ch for ch in text if ch.isalnum())          # Eliminate non-alphanumeric characters
    city_text = [city_text[i:i + 3] for i in range(0, len(city_text), 3)]       # Create list of city codes
                                                                    
    return city_text

#### Function: Returns the issued date range for the Travel Waiver

In [6]:
from datetime import datetime, timedelta
import calendar

# List Comphrension that creates tuple of each month number and name to match against the waiver's dates
# [()] then converted to {}
month_names = [(calendar.month_name[month], n + 1) for n, month in enumerate(range(1,13))]
month_names = dict(month_names)


def get_dates(text):
    '''
    This function takes the substring from the string_search() function
    to get the Travel Waiver's issued date range
    An example of a substring: '(s): December 1, 2019  *     '
    This function will return the start and end date, along with
    a list of every date in that range, all in YYYY-mm-dd format
    '''
    
    months = [m for m in month_names if m in text]             # Travel Waiver's month(s) as string
    day_and_year = list(map(int, re.findall(r'\d+', text)))    # Regex to find the numeric values in substring
    year = day_and_year[-1]                                    # Travel Waiver's Year 

    # Condition for Travel Waivers spanning multiple days - example: 'August 25 - 27'
    if len(day_and_year) == 3 and len(months) == 1:
        mnth = int(month_names[months[0]])
        start_day = day_and_year[0]                   # Starting day
        end_day = day_and_year[1]                     # Ending day

        day_range = (datetime(year, mnth, end_day) - datetime(year, mnth, start_day)).days
        date_range_list = [(datetime(year, mnth, start_day) + \
                    timedelta(days = d)).strftime('%Y-%m-%d') for d in range(day_range + 1)]
        start_date = date_range_list[0]                                 # Start date in the date range
        end_date = date_range_list[-1]                                  # Last date in the date range

        
    # Condition for Travel Waivers spanning multiple months - example: 'September 30 – October 2, 2019'
    elif len(day_and_year) == 3 and len(months) == 2:
        start_month = int(month_names[months[0]])      # Get month number of starting month 
        end_month = int(month_names[months[1]])        # Get month number of ending month
        start_day = day_and_year[0]                    # Starting day
        end_day = day_and_year[1]                      # Ending day
                
        day_range = (datetime(year, end_month, end_day) - datetime(year, start_month, start_day)).days
        date_range_list = [(datetime(year, start_month, start_day) + \
                            timedelta(days = d)).strftime('%Y-%m-%d') for d in range(day_range + 1)]
        start_date = date_range_list[0]                                  # Start date in the date range
        end_date = date_range_list[-1]                                   # Last date in the date range

        
    # Condition for Travel Waiver that is valid for a single day
    else: 
        date_range_list = datetime(year, int(month_names[months[0]]), day_and_year[0]).strftime('%Y-%m-%d')
        start_date = date_range_list
        end_date = date_range_list
        
        
    return start_date, end_date, date_range_list

# Iterate over Dictionary to Create the Cleaned Dataset

#### The functions above are called in the iteration below to extract and structure the data 

In [7]:
for value in travel_waivers:
    '''
    Iterate over the Travel Waiver Dictionary, calling the functions
    above to clean and structure the data for output
    '''
    
    # Remove the '\r\n' in the email body text
    body_text = travel_waivers[value]['email_body']
    body_text = body_text.replace('\n', ' ').replace('\r', '')

    # Call 'clean_title()' function 
    title_text = travel_waivers[value]['email_title']
    event, indicator = clean_title(title_text)
    
    # Call 'string_search()' function to extract the cities and dates text from the Travel Waiver
    cities = string_search(body_text, 'c', 'c')
    dates = string_search(body_text, 'd', 'd')
    
    # Call 'clean_cities()' function to create list of city codes from the Travel Waiver
    city_list = clean_cities(cities)
    
    # Call 'get_dates()' function to get list of dates (YYYY-mm-dd) associated with the Travel Waiver 
    start, end, all_dates = get_dates(dates)
    
    # Update dictionary values
    travel_waivers[value]['Event'] = event
    travel_waivers[value]['Indicator'] = indicator
    travel_waivers[value]['email_body'] = body_text
    travel_waivers[value]['City'] = city_list
    travel_waivers[value]['Start_Date'] = start
    travel_waivers[value]['End_Date'] = end
    travel_waivers[value]['All_Dates'] = all_dates

# Prepare Final Data for Output

In [8]:
# Convert the Dictionary with the data to DataFrame
travel_waiver_data = pd.DataFrame.from_dict(travel_waivers, orient = 'index')

# Drop the columns that contained the original scraped date ['email_title', 'email_body']
travel_waiver_data = travel_waiver_data[[
    'Issue_Date', 
    'Event',
    'Indicator', 
    'City',
    'Start_Date',
    'End_Date',
    'All_Dates'
    ]]

In [9]:
# explode() function takes a DataFrame column and creates a new row for each item  
travel_waiver_data_expanded = travel_waiver_data.explode('City')
travel_waiver_data_expanded = travel_waiver_data_expanded.explode('All_Dates')
travel_waiver_data_expanded = travel_waiver_data_expanded.drop_duplicates().reset_index(drop = True)

#### Save DataFrame as .txt file

In [10]:
today = datetime.now().strftime('%Y%m%d')
save_path = 'travel_waivers_' + today + '.csv'
travel_waiver_data_expanded.to_csv(save_path, index = False)

In [11]:
travel_waiver_data_expanded.tail(15)

,Issue_Date,Event,Indicator,City,Start_Date,End_Date,All_Dates
1963,2018-12-25,Northern Plains Winter Weather,SEV1+,FAR,2018-12-26,2018-12-27,2018-12-27
1964,2018-12-25,Northern Plains Winter Weather,SEV1+,FSD,2018-12-26,2018-12-27,2018-12-26
1965,2018-12-25,Northern Plains Winter Weather,SEV1+,FSD,2018-12-26,2018-12-27,2018-12-27
1966,2018-12-25,Northern Plains Winter Weather,SEV1+,ISN,2018-12-26,2018-12-27,2018-12-26
1967,2018-12-25,Northern Plains Winter Weather,SEV1+,ISN,2018-12-26,2018-12-27,2018-12-27
1968,2018-12-25,Northern Plains Winter Weather,SEV1+,JMS,2018-12-26,2018-12-27,2018-12-26
1969,2018-12-25,Northern Plains Winter Weather,SEV1+,JMS,2018-12-26,2018-12-27,2018-12-27
1970,2018-12-25,Northern Plains Winter Weather,SEV1+,MOT,2018-12-26,2018-12-27,2018-12-26
1971,2018-12-25,Northern Plains Winter Weather,SEV1+,MOT,2018-12-26,2018-12-27,2018-12-27
1972,2018-12-25,Northern Plains Winter Weather,SEV1+,RAP,2018-12-26,2018-12-27,2018-12-26
